In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session



In [ ]:
#read excel
example_df = pd.read_excel('example_datatable_final.xlsx')

In [ ]:
example_df

In [4]:
import re
import sqlite3

def get_create_table_string(tablename, connection):
    sql = """
    select * from sqlite_master where name = "{}" and type = "table"
    """.format(tablename) 
    result = connection.execute(sql)

    create_table_string = result.fetchmany()[0][4]
    return create_table_string

def add_pk_to_create_table_string(create_table_string, colname):
    regex = "(\n.+{}[^,]+)(,)".format(colname)
    return re.sub(regex, "\\1 PRIMARY KEY,",  create_table_string, count=1)

def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

In [6]:
# # df = pd.DataFrame({"a": [1,2,3], "b": [2,3,4]})
# con = sqlite3.connect("deleteme.db")
# df.to_sql("df", con, if_exists="replace")

# add_pk_to_sqlite_table("df", "index", con)
# r = con.execute("select sql from sqlite_master where name = 'df' and type = 'table'")
# print(r.fetchone()[0])

con = sqlite3.connect("HCMST_all_wvs_fake.db")

example_df.to_sql("api_data_table", con, if_exists="replace")

add_pk_to_sqlite_table("api_data_table", "index", con)
r = con.execute("select sql from sqlite_master where name = 'api_data_table' and type = 'table'")
print(r.fetchone()[0])


CREATE TABLE "api_data_table" (
"index" INTEGER PRIMARY KEY,
  "caseid_new" INTEGER,
  "qflag" TEXT,
  "married" TEXT,
  "children_in_hh" INTEGER,
  "age" INTEGER,
  "age_bin" TEXT,
  "age_difference_bin" TEXT,
  "age_difference" INTEGER,
  "same_sex_couple" TEXT,
  "met_online" INTEGER,
  "how_long_relationship" INTEGER,
  "relationship_quality" TEXT,
  "how_met" TEXT
)
